In [1]:
# imports and device setup

# Standard library
import os
import time

# for data handling and images
import numpy as np
import pandas as pd
from PIL import Image

# PyTorch and TorchVision for deep learning
import torch
import torchvision
import torchvision.transforms as T

# YOLO model from the Ultralytics library
from ultralytics import YOLO

# Configure computation device:
# Use GPU if available, otherwise fall back to CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Brandon\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Using device: cpu


In [3]:
# load pre-trained models

# YOLO from ultralytics
yolo_model = YOLO("yolov8n.pt")
yolo_model.to(device)

# Faster R-CNN from torchvision with default COCO weights
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights

weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
fasterrcnn_model = fasterrcnn_resnet50_fpn(weights=weights)
fasterrcnn_model.to(device)

# Set model to evaluation mode
fasterrcnn_model.eval()

print("Models loaded.")


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\Brandon/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|███████████████████████████████████████████████████████████████████████████████| 160M/160M [00:09<00:00, 18.2MB/s]


Models loaded.


In [5]:
# helper functions

transform = T.Compose([T.ToTensor()])

def run_yolo(image_path, model, device=device, conf_threshold=0.25):
    """
    Runs YOLO on a single image and returns:
    - inference time (seconds)
    - number of detected objects
    - average confidence score
    """
    start = time.time()
    results = model(image_path, conf=conf_threshold, verbose=False, device=device)
    end = time.time()
    
    r = results[0]
    boxes = r.boxes  # bounding boxes + scores
    
    num_objects = len(boxes)
    if num_objects > 0:
        probs = boxes.conf.detach().cpu().numpy()
        avg_prob = float(probs.mean())
    else:
        avg_prob = 0.0
    
    elapsed = end - start
    return elapsed, num_objects, avg_prob


def run_fasterrcnn(image_path, model, device=device, score_threshold=0.5):
    """
    Runs Faster R-CNN on a single image and returns:
    - inference time (seconds)
    - number of detected objects with score >= threshold
    - average score for detections kept
    """
    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).to(device)
    
    start = time.time()
    with torch.no_grad():
        outputs = model([img_tensor])
    end = time.time()
    
    output = outputs[0]
    scores = output["scores"].detach().cpu().numpy()
    
    keep = scores >= score_threshold
    num_objects = int(keep.sum())
    avg_prob = float(scores[keep].mean()) if num_objects > 0 else 0.0
    
    elapsed = end - start
    return elapsed, num_objects, avg_prob


def get_basic_image_features(image_path):
    """
    Extracts simple, non-deep-learning image properties:
    - width, height
    - aspect ratio
    - mean brightness (0-1)
    """
    img = Image.open(image_path).convert("RGB")
    width, height = img.size
    
    arr = np.array(img).astype("float32") / 255.0
    brightness = float(arr.mean())
    aspect_ratio = width / height
    
    return width, height, aspect_ratio, brightness


In [6]:
# image folder and file list

# Folder containing the test images for model comparison
image_folder = r"C:\Users\Brandon\Documents\MSBA\MSBA_in_progress_courses\MSBA_503_Analytics_programming_2\Take Home Assignment\Images"

# Manually defined image selection representing different use-case scenarios
image_files = [
    "bike.jpg",
    "breakdown.jpg",
    "camp.jpg",
    "jump.jpg",
    "parts.jpg",
    "propane.jpg",
    "repairs.jpg",
    "rocks.jpg",
    "tow.jpg",
    "water.jpg"
]

# Construct full file paths for downstream processing
image_paths = [os.path.join(image_folder, f) for f in image_files]

# Confirm that all expected images are present and readable
for p in image_paths:
    print(p, "exists:", os.path.exists(p))


C:\Users\Brandon\Documents\MSBA\MSBA_in_progress_courses\MSBA_503_Analytics_programming_2\Take Home Assignment\Images\bike.jpg exists: True
C:\Users\Brandon\Documents\MSBA\MSBA_in_progress_courses\MSBA_503_Analytics_programming_2\Take Home Assignment\Images\breakdown.jpg exists: True
C:\Users\Brandon\Documents\MSBA\MSBA_in_progress_courses\MSBA_503_Analytics_programming_2\Take Home Assignment\Images\camp.jpg exists: True
C:\Users\Brandon\Documents\MSBA\MSBA_in_progress_courses\MSBA_503_Analytics_programming_2\Take Home Assignment\Images\jump.jpg exists: True
C:\Users\Brandon\Documents\MSBA\MSBA_in_progress_courses\MSBA_503_Analytics_programming_2\Take Home Assignment\Images\parts.jpg exists: True
C:\Users\Brandon\Documents\MSBA\MSBA_in_progress_courses\MSBA_503_Analytics_programming_2\Take Home Assignment\Images\propane.jpg exists: True
C:\Users\Brandon\Documents\MSBA\MSBA_in_progress_courses\MSBA_503_Analytics_programming_2\Take Home Assignment\Images\repairs.jpg exists: True
C:\Users

In [7]:
# run YOLO and Faster R-CNN on all images

# Storage container for model performance and basic image metrics
rows = []

for img_name, img_path in zip(image_files, image_paths):
    print(f"\nProcessing {img_name} ...")
    
    # non-deep-learning features
    width, height, aspect, brightness = get_basic_image_features(img_path)
    
    # YOLO
    # Returns inference time, number of detections, and average confidence
    y_time, y_count, y_avg = run_yolo(img_path, yolo_model)
    print(f"  YOLO    -> time: {y_time:.3f}s, objects: {y_count}, avg score: {y_avg:.3f}")
    
    # Faster R-CNN
    # Performance comparison with a slower, two-stage detector
    f_time, f_count, f_avg = run_fasterrcnn(img_path, fasterrcnn_model)
    print(f"  FasterRCNN -> time: {f_time:.3f}s, objects: {f_count}, avg score: {f_avg:.3f}")

    # Store all results for later analysis and export
    rows.append({
        "image": img_name,
        "width": width,
        "height": height,
        "aspect_ratio": aspect,
        "mean_brightness": brightness,
        "yolo_time_sec": y_time,
        "yolo_objects": y_count,
        "yolo_avg_score": y_avg,
        "fasterrcnn_time_sec": f_time,
        "fasterrcnn_objects": f_count,
        "fasterrcnn_avg_score": f_avg,
    })

# Convert results into a DataFrame for easy review and CSV export
results_df = pd.DataFrame(rows)
results_df



Processing bike.jpg ...
  YOLO    -> time: 0.285s, objects: 3, avg score: 0.467
  FasterRCNN -> time: 2.510s, objects: 13, avg score: 0.798

Processing breakdown.jpg ...
  YOLO    -> time: 0.086s, objects: 7, avg score: 0.593
  FasterRCNN -> time: 2.109s, objects: 9, avg score: 0.832

Processing camp.jpg ...
  YOLO    -> time: 0.169s, objects: 5, avg score: 0.441
  FasterRCNN -> time: 2.202s, objects: 19, avg score: 0.748

Processing jump.jpg ...
  YOLO    -> time: 0.120s, objects: 2, avg score: 0.533
  FasterRCNN -> time: 2.083s, objects: 3, avg score: 0.923

Processing parts.jpg ...
  YOLO    -> time: 0.056s, objects: 1, avg score: 0.821
  FasterRCNN -> time: 2.049s, objects: 3, avg score: 0.633

Processing propane.jpg ...
  YOLO    -> time: 0.092s, objects: 3, avg score: 0.562
  FasterRCNN -> time: 2.049s, objects: 6, avg score: 0.742

Processing repairs.jpg ...
  YOLO    -> time: 0.055s, objects: 5, avg score: 0.519
  FasterRCNN -> time: 2.033s, objects: 9, avg score: 0.799

Proce

,image,width,height,aspect_ratio,mean_brightness,yolo_time_sec,yolo_objects,yolo_avg_score,fasterrcnn_time_sec,fasterrcnn_objects,fasterrcnn_avg_score
0,bike.jpg,3651,5477,0.666606,0.278522,0.285437,3,0.466808,2.510209,13,0.798325
1,breakdown.jpg,768,512,1.500000,0.552281,0.085668,7,0.592944,2.108643,9,0.831561
2,camp.jpg,3651,5477,0.666606,0.483322,0.169159,5,0.440967,2.201861,19,0.747787
3,jump.jpg,3651,5477,0.666606,0.553946,0.119543,2,0.533195,2.082830,3,0.922694
4,parts.jpg,768,512,1.500000,0.558088,0.056018,1,0.820582,2.049387,3,0.632742
5,propane.jpg,2500,3748,0.667022,0.429364,0.092395,3,0.561820,2.048584,6,0.741773
6,repairs.jpg,768,512,1.500000,0.577062,0.055180,5,0.519464,2.033381,9,0.798710
7,rocks.jpg,5477,3651,1.500137,0.529405,0.150095,2,0.775463,2.085133,2,0.964359
8,tow.jpg,2500,1668,1.498801,0.577895,0.097066,3,0.764260,2.078954,4,0.991021
9,water.jpg,3000,2001,1.499250,0.505922,0.111621,1,0.682438,2.038385,4,0.758816


In [8]:
# save results to CSV

# Output file name and location
output_csv = os.path.join(image_folder, "object_detection_results.csv")

# Save the DataFrame containing performance metrics
results_df.to_csv(output_csv, index=False)

print("Saved results to:", output_csv)


Saved results to: C:\Users\Brandon\Documents\MSBA\MSBA_in_progress_courses\MSBA_503_Analytics_programming_2\Take Home Assignment\Images\object_detection_results.csv
